<a href="https://colab.research.google.com/github/Madjogger1202/JamSpelL_ru/blob/main/Spell_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Тестовый запуск

In [1]:
!git clone https://huggingface.co/datasets/Madjogger/JamSpell_dataset
!git clone https://github.com/Madjogger1202/JamSpell.git
!cd JamSpell
!cmake /content/JamSpell/CMakeLists.txt
!make
!./main/jamspell train /content/JamSpell/alphabet/en_ascii.txt /content/JamSpell_dataset/eng.txt en_model.bin
!sudo apt-get install swig3.0
!sudo pip install jamspell
!sudo pip install  autocorrect
!sudo pip install pyspellchecker
! pip install bitarray

Cloning into 'JamSpell_dataset'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 25 (delta 6), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (25/25), 3.36 KiB | 860.00 KiB/s, done.
Filtering content: 100% (6/6), 2.58 GiB | 75.70 MiB/s, done.
Cloning into 'JamSpell'...
remote: Enumerating objects: 1200, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 1200 (delta 82), reused 47 (delta 19), pack-reused 1022
Receiving objects: 100% (1200/1200), 712.88 KiB | 16.20 MiB/s, done.
Resolving deltas: 100% (723/723), done.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- 

Создать побитый файл - 12% бит искажены

In [6]:
import os
import random
import binascii
from bitarray import bitarray

err_percent = 12
start_txt = "I'm so tired after a long day at work. \nI need to get some sleep."
space_count=start_txt.count(' ')
print(space_count)
out_txt=''
out_txt+=chr(space_count)
out_txt+=chr(space_count)
out_txt+=chr(space_count)
curr_pos=0
for i in start_txt:
  if(i==' '):
    print(curr_pos)
    out_txt+=chr(curr_pos)
    out_txt+=chr(curr_pos)
    out_txt+=chr(curr_pos)
  curr_pos+=1
print(out_txt)
if os.path.exists('/content/spaces.txt') != 0:
  os.remove("/content/spaces.txt")
f = open('/content/spaces.txt','w')
f.write(out_txt)
f.close()

def set_bit(v, index, x):
  """Set the index:th bit of v to 1 if x is truthy, else to 0, and return the new value."""
  mask = 1 << index   # Compute mask, an integer with just bit 'index' set.
  v &= ~mask          # Clear the bit indicated by the mask (if x is False)
  if x:
    v |= mask         # If x was True, set the bit indicated by the mask.
  return v            # Return the result, we're done.
def to_bin_s(a):
    c =' '.join(['0'*(8-len(bin(ord(i))[2:]))+(bin(ord(i))[2:]) for i in a])
    return c

def bits(ch_in, pos):
  in_byte=to_bin_s(ch_in)
  print(in_byte)
  return (in_byte[pos]=='1')

recieved_str=''
for i in start_txt:
  print(bin(ord(i))[2:].zfill(8))
  bts = bitarray(bin(ord(i))[2:].zfill(8))
  print(bts.tobytes().decode('ascii'))
print('Out:')
for i in start_txt:
  out_char=(bin(ord(i))[2:].zfill(8))
  str_bin_ch=''
  for j in range(0, 8):
    bad_symbol=(random.randint(0, 1000)<=err_percent)
    if(bad_symbol):
      str_bin_ch+='1'
      print('bad!')
    else:
      str_bin_ch+=out_char[j]
      #set_bit(out_char, j, 0)
  bts = bitarray(str_bin_ch)
  recieved_str+=bts.tobytes().decode('ascii')
  print(str_bin_ch)
  print(recieved_str)

print(recieved_str)
if os.path.exists('/content/recieved_str.txt') != 0:
  os.remove("/content/recieved_str.txt")
f = open('/content/recieved_str','w')
f.write(recieved_str)
f.close()

14
3
6
12
18
20
25
29
32
38
41
46
49
53
58
   &&&)))...111555:::
01001001
I
00100111
'
01101101
m
00100000
 
01110011
s
01101111
o
00100000
 
01110100
t
01101001
i
01110010
r
01100101
e
01100100
d
00100000
 
01100001
a
01100110
f
01110100
t
01100101
e
01110010
r
00100000
 
01100001
a
00100000
 
01101100
l
01101111
o
01101110
n
01100111
g
00100000
 
01100100
d
01100001
a
01111001
y
00100000
 
01100001
a
01110100
t
00100000
 
01110111
w
01101111
o
01110010
r
01101011
k
00101110
.
00100000
 
00001010


01001001
I
00100000
 
01101110
n
01100101
e
01100101
e
01100100
d
00100000
 
01110100
t
01101111
o
00100000
 
01100111
g
01100101
e
01110100
t
00100000
 
01110011
s
01101111
o
01101101
m
01100101
e
00100000
 
01110011
s
01101100
l
01100101
e
01100101
e
01110000
p
00101110
.
Out:
01001001
I
00100111
I'
01101101
I'm
00100000
I'm 
01110011
I'm s
01101111
I'm so
00100000
I'm so 
01110100
I'm so t
01101001
I'm so ti
01110010
I'm so tir
01100101
I'm so tire
01100100
I'm so

Исправление ошибок:

In [8]:
import jamspell
import time
from autocorrect import Speller
from collections import Counter
from array import array

spell = Speller(lang='en')
jsp = jamspell.TSpellCorrector()
jsp.LoadLangModel('en_model.bin')

good_symbols_en = " \abcdefghijklmnopqrstuvwxyz!,.0123456789?'ABCDEFGHIJKLMNOPQRSTUVWXYZ"
bad_mid_symbols_en = "!,.0123456789?%:;+=-()/?[]|"
bad_start_symbols_en = "!,.0123456789?%:;+=-()/?[]|"
bad_end_symbols_en = "!0123456789?%:;+=-()/?[]|"
num_symbols = "0123456789"
letters_symbols_en = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
extra_symbols_en = "'"

f = open('/content/recieved_str','r')
input_str = f.read()
print(input_str)
input_str=input_str.replace(" ", ".")

f.close()

f = open('/content/spaces.txt','r')
input_spaces = f.read()
f.close()

print(jsp.FixFragment(input_str))

#fix spaces:
def most_comm(ch1, ch2, ch3):
  str1=ch1+ch2+ch3
  count = sorted([(str1.count(letter), letter) for letter in set(str1)], reverse=True)
  return count[0][1]

list_str1=list(input_str)
num_spaces = ord(most_comm(input_spaces[0], input_spaces[1], input_spaces[2]))
print(num_spaces)
for i in range(1, num_spaces+1):
  list_str1[ord(most_comm(input_spaces[i*3], input_spaces[i*3+1], input_spaces[i*3+2]))]=' '

start = time.time()


for i in range(0, len(list_str1)-1):
  if list_str1[i] not in good_symbols_en:
    list_str1[i]='/'
  if list_str1[i] in num_symbols:
    if i>=1 and i<len(list_str1):
      if (list_str1[i-1] in letters_symbols_en) or (list_str1[i+1] in letters_symbols_en):
        list_str1[i]='/'
  if list_str1[i] in bad_mid_symbols_en:
    if i>=1 and i<len(list_str1):
      if (list_str1[i-1] != ' ') or (list_str1[i+1] != ' '):
        list_str1[i]='/'

new_str1=''.join(list_str1)

print(new_str1)


str_txt1 = jsp.FixFragment(new_str1)
print(str_txt1)

new_txt1=''
isNewW=True
isFirstUpper=False
for i in str_txt1:
  if(isNewW):
    isFirstUpper=i.isupper()
    isNewW=False
  elif (i.isupper() and (not isNewW) and (not isFirstUpper)):
    i=i.lower()
  if(i==' '):
    isNewW=True
  new_txt1+=i

str_txt1=new_txt1
str_final = spell(new_txt1)
print(str_final)

end = time.time()
print("jamspell обработал за : ",
      (end-start) * 10**3, "ms")
my_file = open("/content/output_text.txt", "w+")
my_file.write(str_final)
my_file.close()

I'm so tired after a long day at work. 
I need`|o get some wlemp.
I'm.so.tired.after.a.long.day.at.work..
I.need`|o.get.some.wlemp.
14
I'm so tired /fter / long d/y /t work/ /I need /o get some wlemp.
I'm so tired After A long dAy At work I need To get some wlemp.
I'm so tired After A long day At work I need To get some sleep.
jamspell обработал за :  35.88128089904785 ms


# Training model

##Download text dataset and source code

In [ ]:
!git clone https://huggingface.co/datasets/Madjogger/JamSpell_dataset

In [2]:
!git clone https://github.com/Madjogger1202/JamSpell.git

Cloning into 'JamSpell'...
remote: Enumerating objects: 1200, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 1200 (delta 82), reused 47 (delta 19), pack-reused 1022
Receiving objects: 100% (1200/1200), 712.91 KiB | 13.71 MiB/s, done.
Resolving deltas: 100% (723/723), done.


##Build jamspell

In [3]:
!cd JamSpell
!cmake /content/JamSpell/CMakeLists.txt
!make

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find GTest (missing: GTEST_LIBRARY GTEST_INCLUDE_DIR GTEST_MAIN_LIBRARY) 
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE  
-- Configuring done (1.1s)
-- Generating done (0.0s)
-- Build files have been written to: /content
[  4%] Building CXX object contrib/cityhash/CMakeFiles/cityhash.dir/city.cc.o
[  8%] Linking CXX static library libcityhash.a
[  8%] Built target cityhash
[ 12%] Building CXX object contri

##Train model - check folders for alphabet and dataset
6 languages supported now, but you can create your own:

1.Download or parse from somewhere data on your language (books, web sites, wikipedia). Useful resource: https://wortschatz.uni-leipzig.de/en/download (data for many languages), from here you can use files, named sentences

2.Create txt with alphabet symbols and (optionally) extra symbol to deal with errors

In [4]:
!./main/jamspell train /content/JamSpell/alphabet/en_ascii.txt /content/JamSpell_dataset/eng.txt en_model.bin

[info] loading text
[info] generating N-grams 3303778
[info] processed 3.90992%
[info] processed 8.01827%
[info] processed 11.5307%
[info] processed 15.9245%
[info] processed 19.3134%
[info] processed 23.0437%
[info] processed 26.562%
[info] processed 31.1124%
[info] processed 34.938%
[info] processed 39.1535%
[info] processed 40.9671%
[info] processed 44.9287%
[info] processed 48.7293%
[info] processed 52.8333%
[info] processed 56.5897%
[info] processed 59.2908%
[info] processed 62.9202%
[info] processed 66.3844%
[info] processed 69.8103%
[info] processed 73.0434%
[info] processed 76.4221%
[info] processed 79.9587%
[info] processed 83.3144%
[info] processed 86.1689%
[info] processed 87.5265%
[info] processed 90.8772%
[info] processed 94.6782%
[info] processed 97.9386%
[info] processed 99.9155%
[info] generating keys
[info] ngrams1: 615465
[info] ngrams2: 9788693
[info] ngrams3: 27806369
[info] total: 38210527
[info] generating perf hash
[info] finished, buckets: 47763197
[info] bucket

##Load for first time
It can take some time to generate bin.spell file, once it is generated, model will load very fast

In [5]:
!sudo apt-get install swig3.0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig3.0
0 upgraded, 1 newly installed, 0 to remove and 18 not upgraded.
Need to get 1,109 kB of archives.
After this operation, 5,555 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 swig3.0 amd64 3.0.12-2.2ubuntu1 [1,109 kB]
Fetched 1,109 kB in 0s (8,131 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package 

In [6]:
!sudo pip install jamspell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jamspell: filename=jamspell-0.0.12-cp310-cp310-linux_x86_64.whl size=1782250 sha256=c489e1e3438f6fe66683ea6ce95d30b1496906ffb46cf6e83a7db80386082c73
  Stored in directory: /root/.cache/pip/wheels/95/af/d3/fb57122fda2851ad5972ce80640001c568dcffdf16bc9ddb17
Successfully built jamspell


In [7]:
import jamspell

In [9]:
jsp = jamspell.TSpellCorrector()
jsp.LoadLangModel('en_model.bin')
str_txt = jsp.FixFragment("I'm so tierd after ta lvng day at wonk, I newed to get som sleeep.")

print(str_txt)

I'm so tired after a long day at work, I need to get some sleep.


# Post processing and tests
Correct upper-lower letters if it is nessesary

In [142]:
!sudo pip install  autocorrect
!sudo pip install pyspellchecker

ERROR: Operation cancelled by user


In [143]:
import jamspell
import time
from autocorrect import Speller
spell = Speller(lang='en')

jsp = jamspell.TSpellCorrector()
jsp.LoadLangModel('en_model.bin')

good_symbols_en = " abcdefghijklmnopqrstuvwxyz!,.0123456789?'ABCDEFGHIJKLMNOPQRSTUVWXYZ"
bad_mid_symbols_en = "!,.0123456789?%:;+=-()/?[]\|"
bad_start_symbols_en = "!,.0123456789?%:;+=-()/?[]\|"
bad_end_symbols_en = "!,.0123456789?%:;+=-()/?[]\|"
num_symbols = "0123456789"
letters_symbols_en = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
extra_symbols_en = "'"

str1='Y~u crn gen35ate ext^nsions for other langu#ges !'
str2="I'm so tirel a&ter a |ong Tay at work: [need to get some s,eep."
str_txt1 = jsp.FixFragment(str1)
str_txt2 = jsp.FixFragment(str2)
print(str_txt1)
print(str_txt2)
start = time.time()
list_str1=list(str1)
list_str2=list(str2)

for i in range(0, len(list_str1)-1):
  if list_str1[i] not in good_symbols_en:
    list_str1[i]='/'
  if list_str1[i] in num_symbols:
    if i>=1 and i<len(list_str1):
      if (list_str1[i-1] in letters_symbols_en) or (list_str1[i+1] in letters_symbols_en):
        list_str1[i]='/'
  if list_str1[i] in bad_mid_symbols_en:
    if i>=1 and i<len(list_str1):
      if (list_str1[i-1] != ' ') or (list_str1[i+1] != ' '):
        list_str1[i]='/'

for i in range(0, len(list_str2)-1):
  if list_str2[i] not in good_symbols_en:
    list_str2[i]='/'
  if list_str2[i] in num_symbols:
    if i>=1 and i<len(list_str2):
      if (list_str2[i-1] in letters_symbols_en) or (list_str2[i+1] in letters_symbols_en):
        list_str2[i]='/'
  if list_str2[i] in bad_mid_symbols_en:
    if i>=1 and i<len(list_str2):
      if not ((list_str2[i-1] == ' ') and (list_str2[i+1] == ' ')):
        list_str2[i]='/'


new_str1=''.join(list_str1)
new_str2=''.join(list_str2)
str_test3=new_str2

print(new_str1)
print(new_str2)

str_txt1 = jsp.FixFragment(new_str1)
str_txt2 = jsp.FixFragment(new_str2)
print(str_txt1)
print(str_txt2)

new_txt1=''
isNewW=True
isFirstUpper=False
for i in str_txt1:
  if(isNewW):
    isFirstUpper=i.isupper()
    isNewW=False
  elif (i.isupper() and (not isNewW) and (not isFirstUpper)):
    i=i.lower()
  if(i==' '):
    isNewW=True
  new_txt1+=i

new_txt2=''
isNewW=True
isFirstUpper=False
for i in str_txt2:
  if(isNewW):
    isFirstUpper=i.isupper()
    isNewW=False
  elif (i.isupper() and (not isNewW) and (not isFirstUpper)):
    i=i.lower()
  if(i==' '):
    isNewW=True
  new_txt2+=i

str_txt1=new_txt1
str_txt2=new_txt2

print(spell(new_txt1))
print(spell(new_txt2))

end = time.time()
print("jamspell обработал за : ",
      (end-start) * 10**3, "ms")

Y~u crn gen35ate ext^nsions for other langu#ges !
I'm so tired a&ter a |on May at work: [need to get some s,eep.
Y/u crn gen//ate ext/nsions for other langu/ges !
I'm so tirel a/ter a /ong Tay at work/ /need to get some s/eep.
YOu can genERate extEnsions for other languAges !
I'm so tired aFter a Long Day at work Need to get some sLeep.
You can generate extensions for other languages !
I'm so tired after a Long Day at work Need to get some sleep.
jamspell обработал за :  6.057262420654297 ms


If you want to correct most wrong uppercases just try this:

Also test out how autocorrect works alone:

In [144]:
spell('Y~u crn gen35ate ext^nsions for other langu#ges !')

'Y~u can gen35ate ext^signs for other lang#ges !'

In [145]:
import re
from spellchecker import SpellChecker
english = SpellChecker(language='en')
print(str_test3)
words=re.findall(r"[\w']+|[.,!?;]", "str_test3")
#words = new_txt2.split()
start = time.time()
misspelled=['Y~u', 'crn', 'gen35ate', 'ext^nsions', 'for', 'other', 'langu#ges', '!']
for word in words:
    # Get the one `most likely` answer
    print(english.correction(word))
end = time.time()
print("pyspellchecker обработал за : ",
      (end-start) * 10**3, "ms")

I'm so tirel a/ter a /ong Tay at work/ /need to get some s/eep.
None
pyspellchecker обработал за :  235.5494499206543 ms


In [146]:
import jamspell
jsp = jamspell.TSpellCorrector()
assert jsp.LoadLangModel('/content/big_model2_eng.bin')
str_txt = jsp.FixFragment('Y~u crn gen35ate ext^nsions for other langu#ges !')
# u'I am the best spell checker!'
start = time.time()
new_txt=''
isNewW=True
isFirstUpper=False
for i in str_txt:
  if(isNewW):
    isFirstUpper=i.isupper()
    isNewW=False
  elif (i.isupper() and (not isNewW) and (not isFirstUpper)):
    i=i.lower()
  if(i==' '):
    isNewW=True
  new_txt+=i
print(new_txt)
from autocorrect import Speller
spell = Speller(lang='en')
print(spell(new_txt))
end = time.time()
print("jamspell обработал за : ",
      (end-start) * 10**3, "ms")

AssertionError: ignored

In [ ]:
import sys
#os.environ["PYTHONIOENCODING"] = "Win-1251"

str_alp=''
for i in range(0, 255, 1):
  str_alp+=(chr(i))
print(ascii(str_alp))

#Generate noise


In [70]:
! pip install bitarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.5/286.5 kB 7.1 MB/s eta 0:00:00


In [117]:
import os
import random
import binascii
from bitarray import bitarray

err_percent = 12
start_txt = "I'm so tired after a long day at work. \nI need to get some sleep."
space_count=start_txt.count(' ')
print(space_count)
out_txt=''
out_txt+=chr(space_count)
out_txt+=chr(space_count)
out_txt+=chr(space_count)
curr_pos=0
for i in start_txt:
  if(i==' '):
    print(curr_pos)
    out_txt+=chr(curr_pos)
    out_txt+=chr(curr_pos)
    out_txt+=chr(curr_pos)
  curr_pos+=1
print(out_txt)
if os.path.exists('/content/spaces.txt') != 0:
  os.remove("/content/spaces.txt")
f = open('/content/spaces.txt','w')
f.write(out_txt)
f.close()

def set_bit(v, index, x):
  """Set the index:th bit of v to 1 if x is truthy, else to 0, and return the new value."""
  mask = 1 << index   # Compute mask, an integer with just bit 'index' set.
  v &= ~mask          # Clear the bit indicated by the mask (if x is False)
  if x:
    v |= mask         # If x was True, set the bit indicated by the mask.
  return v            # Return the result, we're done.
def to_bin_s(a):
    c =' '.join(['0'*(8-len(bin(ord(i))[2:]))+(bin(ord(i))[2:]) for i in a])
    return c

def bits(ch_in, pos):
  in_byte=to_bin_s(ch_in)
  print(in_byte)
  return (in_byte[pos]=='1')

recieved_str=''
for i in start_txt:
  print(bin(ord(i))[2:].zfill(8))
  bts = bitarray(bin(ord(i))[2:].zfill(8))
  print(bts.tobytes().decode('ascii'))
print('Out:')
for i in start_txt:
  out_char=(bin(ord(i))[2:].zfill(8))
  str_bin_ch=''
  for j in range(0, 8):
    bad_symbol=(random.randint(0, 1000)<=err_percent)
    if(bad_symbol):
      str_bin_ch+='1'
      print('bad!')
    else:
      str_bin_ch+=out_char[j]
      #set_bit(out_char, j, 0)
  bts = bitarray(str_bin_ch)
  recieved_str+=bts.tobytes().decode('ascii')
  print(str_bin_ch)
  print(recieved_str)

print(recieved_str)
if os.path.exists('/content/recieved_str.txt') != 0:
  os.remove("/content/recieved_str.txt")
f = open('/content/recieved_str','w')
f.write(recieved_str)
f.close()

14
3
6
12
18
20
25
29
32
38
41
46
49
53
58
   &&&)))...111555:::
01001001
I
00100111
'
01101101
m
00100000
 
01110011
s
01101111
o
00100000
 
01110100
t
01101001
i
01110010
r
01100101
e
01100100
d
00100000
 
01100001
a
01100110
f
01110100
t
01100101
e
01110010
r
00100000
 
01100001
a
00100000
 
01101100
l
01101111
o
01101110
n
01100111
g
00100000
 
01100100
d
01100001
a
01111001
y
00100000
 
01100001
a
01110100
t
00100000
 
01110111
w
01101111
o
01110010
r
01101011
k
00101110
.
00100000
 
00001010


01001001
I
00100000
 
01101110
n
01100101
e
01100101
e
01100100
d
00100000
 
01110100
t
01101111
o
00100000
 
01100111
g
01100101
e
01110100
t
00100000
 
01110011
s
01101111
o
01101101
m
01100101
e
00100000
 
01110011
s
01101100
l
01100101
e
01100101
e
01110000
p
00101110
.
Out:
01001001
I
00100111
I'
01101101
I'm
00100000
I'm 
01110011
I'm s
01101111
I'm so
00100000
I'm so 
01110100
I'm so t
01101001
I'm so ti
01110010
I'm so tir
01100101
I'm so tire
01100100
I'm so

#Fix noise

In [141]:
import jamspell
import time
from autocorrect import Speller
from collections import Counter
from array import array

spell = Speller(lang='en')
jsp = jamspell.TSpellCorrector()
jsp.LoadLangModel('en_model.bin')

good_symbols_en = " \abcdefghijklmnopqrstuvwxyz!,.0123456789?'ABCDEFGHIJKLMNOPQRSTUVWXYZ"
bad_mid_symbols_en = "!,.0123456789?%:;+=-()/?[]|"
bad_start_symbols_en = "!,.0123456789?%:;+=-()/?[]|"
bad_end_symbols_en = "!,.0123456789?%:;+=-()/?[]|"
num_symbols = "0123456789"
letters_symbols_en = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
extra_symbols_en = "'"

f = open('/content/recieved_str','r')
input_str = f.read()
print(input_str)
input_str=input_str.replace(" ", ".")

f.close()

f = open('/content/spaces.txt','r')
input_spaces = f.read()
f.close()

print(jsp.FixFragment(input_str))

#fix spaces:
def most_comm(ch1, ch2, ch3):
  str1=ch1+ch2+ch3
  count = sorted([(str1.count(letter), letter) for letter in set(str1)], reverse=True)
  return count[0][1]

list_str1=list(input_str)
num_spaces = ord(most_comm(input_spaces[0], input_spaces[1], input_spaces[2]))
print(num_spaces)
for i in range(1, num_spaces+1):
  list_str1[ord(most_comm(input_spaces[i*3], input_spaces[i*3+1], input_spaces[i*3+2]))]=' '

start = time.time()


for i in range(0, len(list_str1)-1):
  if list_str1[i] not in good_symbols_en:
    list_str1[i]='/'
  if list_str1[i] in num_symbols:
    if i>=1 and i<len(list_str1):
      if (list_str1[i-1] in letters_symbols_en) or (list_str1[i+1] in letters_symbols_en):
        list_str1[i]='/'
  if list_str1[i] in bad_mid_symbols_en:
    if i>=1 and i<len(list_str1):
      if (list_str1[i-1] != ' ') or (list_str1[i+1] != ' '):
        list_str1[i]='/'

new_str1=''.join(list_str1)

print(new_str1)


str_txt1 = jsp.FixFragment(new_str1)
print(str_txt1)

new_txt1=''
isNewW=True
isFirstUpper=False
for i in str_txt1:
  if(isNewW):
    isFirstUpper=i.isupper()
    isNewW=False
  elif (i.isupper() and (not isNewW) and (not isFirstUpper)):
    i=i.lower()
  if(i==' '):
    isNewW=True
  new_txt1+=i

str_txt1=new_txt1
str_final = spell(new_txt1)
print(str_final)

end = time.time()
print("jamspell обработал за : ",
      (end-start) * 10**3, "ms")
my_file = open("/content/output_text.txt", "w+")
my_file.write(str_final)
my_file.close()

I'm so tired(after a long day!at wro. 
I need"to get some wleep.
I'm.so.tired(after.a.long.day!at.wro..
I.need"to.get.some.sleep.
14
I'm so tired /fter / long d/y /t w/ro/ /I need to get some wleep.
I'm so tired After A long dAy At wRotE I need to get some sleep.
I'm so tired After A long day At wrote I need to get some sleep.
jamspell обработал за :  18.291711807250977 ms
